# Notebook contains summary of adversarial robustness of different model

In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
from datetime import datetime as dt
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.attacks import fgsm, pgd
from utils.utils import plot_images
from models.resnet import ResNet, SparseResNet
from models.wide_resnet import WideResNet, SparseWideResNet

1.7.0
True


In [3]:
from datetime import datetime as dt

def clean_acc(model, loader):
    clean_correct = 0.
    model.eval()
    with torch.no_grad():
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)  
            clean_correct += (model(X).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        acc = round(clean_correct / len(loader.dataset), 4)
        print(f"clean accuracy: {acc}")
    return acc

def adv_acc(model, loader, loss_fn, attack, attack_params):
    model.eval()
    adv_correct = [0] * len(attack_params)
    for i, (name, params) in enumerate(attack_params):
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)
            noise = attack(model, X, y, loss_fn, **params) 
            adv_correct[i] += (model(X+noise).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        print(f"name: {name} accuracy: {(adv_correct[i] / len(loader.dataset)):.4f}")
    return [round(a/len(loader.dataset), 4) for a in adv_correct]


def clean_acc_bad(model, loader):
    clean_correct = 0.
#     model.eval()
    with torch.no_grad():
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)  
            clean_correct += (model(X).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        acc = round(clean_correct / len(loader.dataset), 4)
        print(f"clean accuracy: {acc}")
    return acc

def adv_acc_bad(model, loader, loss_fn, attack_method, attack_params):
#     model.eval()
    adv_correct = [0] * len(attack_params)
    for k, (X, y) in enumerate(loader, 1):
        start = dt.now().replace(microsecond=0)
        X, y = X.to(device), y.to(device)
        for i, p in enumerate(attack_params):
            noise = attack_method(model, X, y, loss_fn, epsilon=p["epsilon"], 
                                  alpha=p["alpha"], num_iter=p["num_iter"])
            adv_correct[i] += (model(X+noise).max(dim=1)[1] == y).sum().item()
        end = dt.now().replace(microsecond=0)
        print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
    print(" " * 50, end="\r")
    acc = [round(a/len(loader.dataset), 4) for a in adv_correct]
    print(acc)
    return acc

def print_table(models, attacks, clean, adv):
    d = {'model': models, 'clean image': clean}
    for i, (name, p) in enumerate(attacks):
        d[name] = [adv[j][i] for j, _ in enumerate(adv)]

    return pd.DataFrame(data=d)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [6]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

ds = CIFAR10('../data', train=False, target_transform=None, download=True, transform=test_transform)
test_loader = DataLoader(ds, 50, shuffle=True)

Files already downloaded and verified


In [7]:
attack1 = {
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 20
}

attack2 = {
    "epsilon": 4/255,
    "alpha": 2/255,
    "num_iter": 20
}

attack3 = {
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 40
}

attack4 = {
    "epsilon": 4/255,
    "alpha": 2/255,
    "num_iter": 40
}

attack1 = ("eps=8,a=2,i=20", attack1)
attack2 = ("eps=4,a=2,i=20", attack2)
attack3 = ("eps=8,a=2,i=40", attack3)
attack4 = ("eps=4,a=2,i=40", attack4)

params20 = [attack1, attack2]
params40 = [attack3, attack4]

## Bad

In [9]:
# Bez model.eval()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=True).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_bad.pt", map_location=device))

attack1s = {
    "id": 0,
    "name": "eps=8,a=2,i=20",
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 20
}

acc = clean_acc_bad(model, test_loader)
acc_adv = adv_acc_bad(model, test_loader, nn.CrossEntropyLoss(), pgd, [attack1s])

clean accuracy: 0.9022                            
[0.3914]                                          


In [14]:
# z model.eval()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=True).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_bad.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, [attack1])

clean accuracy: 0.7375                            
name: eps=8,a=2,i=20 accuracy: 0.0054             


In [8]:
clean20, adv20 = [], []

In [9]:
torch.cuda.empty_cache()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=False).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params20)

clean20.append(acc)
adv20.append(acc_adv)

clean accuracy: 0.8592                            
name: eps=8,a=2,i=20 accuracy: 0.0047             
name: eps=4,a=2,i=20 accuracy: 0.1093             


In [11]:
torch.cuda.empty_cache()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=False).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_sgd_global.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params20)

clean20.append(acc)
adv20.append(acc_adv)

clean accuracy: 0.896                             
name: eps=8,a=2,i=20 accuracy: 0.0000             
name: eps=4,a=2,i=20 accuracy: 0.0021             


In [13]:
torch.cuda.empty_cache()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=True).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_sgd_local.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params20)

clean20.append(acc)
adv20.append(acc_adv)

clean accuracy: 0.9124                            
name: eps=8,a=2,i=20 accuracy: 0.0000             
name: eps=4,a=2,i=20 accuracy: 0.0005             


In [17]:
models_names = [
    "Sparse-WRN global KW Adam", "Sparse-WRN global KW SGD", "Sparse-WRN local KW SGD"
]
results20 = print_table(models_names, params20, clean20, adv20)
results20

,model,clean image,"eps=8,a=2,i=20","eps=4,a=2,i=20"
0,Sparse-WRN global KW Adam,0.8592,0.0047,0.1093
1,Sparse-WRN global KW SGD,0.8960,0.0000,0.0021
2,Sparse-WRN local KW SGD,0.9124,0.0000,0.0005


In [21]:
torch.cuda.empty_cache()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=False, boost=0.0).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_sgd_no_boost.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params20)

clean20.append(acc)
adv20.append(acc_adv)

clean accuracy: 0.9335                            
name: eps=8,a=2,i=20 accuracy: 0.0000             
name: eps=4,a=2,i=20 accuracy: 0.0014             


In [23]:
torch.cuda.empty_cache()
model = SparseWideResNet(depth=28, widen_factor=10, k_winners=0.1, local=False, boost=0.0).to(device)
model.load_state_dict(torch.load("../saved/sparse_wide_resnet_sgd_no_boost_rezero.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params20)

clean20.append(acc)
adv20.append(acc_adv)

clean accuracy: 0.9051                            
name: eps=8,a=2,i=20 accuracy: 0.0000             
name: eps=4,a=2,i=20 accuracy: 0.0021             


In [24]:
models_names = [
    "Sparse-WRN global KW Adam", "Sparse-WRN global KW SGD", 
    "Sparse-WRN local KW SGD", "Sparse-WRN global KW SGD NO BOOST",
    "Sparse-WRN global KW SGD NO BOOST NO REZERO"
]
results20 = print_table(models_names, params20, clean20, adv20)
results20

,model,clean image,"eps=8,a=2,i=20","eps=4,a=2,i=20"
0,Sparse-WRN global KW Adam,0.8592,0.0047,0.1093
1,Sparse-WRN global KW SGD,0.8960,0.0000,0.0021
2,Sparse-WRN local KW SGD,0.9124,0.0000,0.0005
3,Sparse-WRN global KW SGD NO BOOST,0.9335,0.0000,0.0014
4,Sparse-WRN global KW SGD NO BOOST NO REZERO,0.9051,0.0000,0.0021
